# 7. Model Monitoring & Maintenance

This notebook implements model monitoring, maintenance, and retraining pipelines.

## Objectives
1. Monitor model performance
2. Track prediction distribution
3. Collect user feedback
4. Implement retraining pipeline
5. Set up monitoring dashboards

In [ ]:
import numpy as np
import pandas as pd
import json
import joblib
from pathlib import Path
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import warnings
warnings.filterwarnings('ignore')

## 1. Performance Monitoring

In [ ]:
class ModelMonitor:
    def __init__(self, log_dir='../logs'):
        """Initialize model monitor."""
        self.log_dir = Path(log_dir)
        self.log_dir.mkdir(exist_ok=True)
        
        # Initialize metrics storage
        self.metrics_file = self.log_dir / 'metrics.json'
        self.predictions_file = self.log_dir / 'predictions.csv'
        self.feedback_file = self.log_dir / 'feedback.csv'
        
        self._init_storage()
    
    def _init_storage(self):
        """Initialize storage files if they don't exist."""
        if not self.metrics_file.exists():
            with open(self.metrics_file, 'w') as f:
                json.dump({
                    'daily_metrics': [],
                    'alerts': []
                }, f, indent=2)
        
        if not self.predictions_file.exists():
            pd.DataFrame(columns=[
                'timestamp', 'input_text', 'detected_symptoms',
                'predicted_disease', 'confidence', 'response_time'
            ]).to_csv(self.predictions_file, index=False)
        
        if not self.feedback_file.exists():
            pd.DataFrame(columns=[
                'timestamp', 'prediction_id', 'actual_disease',
                'feedback_text', 'rating'
            ]).to_csv(self.feedback_file, index=False)
    
    def log_prediction(self, prediction_data):
        """Log a new prediction."""
        df = pd.DataFrame([{
            'timestamp': datetime.now().isoformat(),
            'input_text': prediction_data['input_text'],
            'detected_symptoms': ','.join(prediction_data['detected_symptoms']),
            'predicted_disease': prediction_data['predicted_disease'],
            'confidence': prediction_data['confidence'],
            'response_time': prediction_data['response_time']
        }])
        
        df.to_csv(self.predictions_file, mode='a', header=False, index=False)
    
    def log_feedback(self, feedback_data):
        """Log user feedback."""
        df = pd.DataFrame([{
            'timestamp': datetime.now().isoformat(),
            'prediction_id': feedback_data['prediction_id'],
            'actual_disease': feedback_data['actual_disease'],
            'feedback_text': feedback_data['feedback_text'],
            'rating': feedback_data['rating']
        }])
        
        df.to_csv(self.feedback_file, mode='a', header=False, index=False)
    
    def compute_daily_metrics(self):
        """Compute daily performance metrics."""
        # Load predictions and feedback
        predictions_df = pd.read_csv(self.predictions_file)
        feedback_df = pd.read_csv(self.feedback_file)
        
        # Convert timestamps
        predictions_df['timestamp'] = pd.to_datetime(predictions_df['timestamp'])
        feedback_df['timestamp'] = pd.to_datetime(feedback_df['timestamp'])
        
        # Get today's data
        today = datetime.now().date()
        today_predictions = predictions_df[
            predictions_df['timestamp'].dt.date == today
        ]
        today_feedback = feedback_df[
            feedback_df['timestamp'].dt.date == today
        ]
        
        # Compute metrics
        metrics = {
            'date': today.isoformat(),
            'total_predictions': len(today_predictions),
            'avg_confidence': float(today_predictions['confidence'].mean()),
            'avg_response_time': float(today_predictions['response_time'].mean()),
            'feedback_count': len(today_feedback),
            'avg_rating': float(today_feedback['rating'].mean() if len(today_feedback) > 0 else 0)
        }
        
        # Update metrics file
        with open(self.metrics_file, 'r+') as f:
            data = json.load(f)
            data['daily_metrics'].append(metrics)
            f.seek(0)
            json.dump(data, f, indent=2)
        
        return metrics
    
    def check_alerts(self, metrics):
        """Check for performance alerts."""
        alerts = []
        
        # Define thresholds
        thresholds = {
            'min_confidence': 0.7,
            'max_response_time': 1.0,
            'min_rating': 3.5
        }
        
        # Check confidence
        if metrics['avg_confidence'] < thresholds['min_confidence']:
            alerts.append({
                'type': 'low_confidence',
                'message': f"Average confidence ({metrics['avg_confidence']:.2f}) below threshold",
                'timestamp': datetime.now().isoformat()
            })
        
        # Check response time
        if metrics['avg_response_time'] > thresholds['max_response_time']:
            alerts.append({
                'type': 'high_latency',
                'message': f"Average response time ({metrics['avg_response_time']:.2f}s) above threshold",
                'timestamp': datetime.now().isoformat()
            })
        
        # Check user satisfaction
        if metrics['feedback_count'] > 0 and metrics['avg_rating'] < thresholds['min_rating']:
            alerts.append({
                'type': 'low_satisfaction',
                'message': f"Average rating ({metrics['avg_rating']:.2f}) below threshold",
                'timestamp': datetime.now().isoformat()
            })
        
        # Update alerts in metrics file
        if alerts:
            with open(self.metrics_file, 'r+') as f:
                data = json.load(f)
                data['alerts'].extend(alerts)
                f.seek(0)
                json.dump(data, f, indent=2)
        
        return alerts

# Initialize monitor
monitor = ModelMonitor()

# Test logging
test_prediction = {
    'input_text': "severe headache and fever",
    'detected_symptoms': ['headache', 'fever'],
    'predicted_disease': 'flu',
    'confidence': 0.85,
    'response_time': 0.2
}

monitor.log_prediction(test_prediction)

test_feedback = {
    'prediction_id': '123',
    'actual_disease': 'flu',
    'feedback_text': "Accurate prediction",
    'rating': 5
}

monitor.log_feedback(test_feedback)

# Compute metrics
metrics = monitor.compute_daily_metrics()
print("Daily Metrics:")
print(json.dumps(metrics, indent=2))

# Check alerts
alerts = monitor.check_alerts(metrics)
if alerts:
    print("\nAlerts:")
    print(json.dumps(alerts, indent=2))

## 2. Visualization Dashboard

In [ ]:
def create_monitoring_dashboard():
    """Create monitoring dashboard with plots."""
    # Load data
    predictions_df = pd.read_csv(monitor.predictions_file)
    feedback_df = pd.read_csv(monitor.feedback_file)
    
    with open(monitor.metrics_file, 'r') as f:
        metrics_data = json.load(f)
    
    metrics_df = pd.DataFrame(metrics_data['daily_metrics'])
    
    # Create dashboard
    plt.style.use('seaborn')
    fig = plt.figure(figsize=(15, 10))
    
    # 1. Confidence Distribution
    plt.subplot(2, 2, 1)
    sns.histplot(data=predictions_df, x='confidence', bins=20)
    plt.title('Prediction Confidence Distribution')
    plt.xlabel('Confidence')
    plt.ylabel('Count')
    
    # 2. Response Time Trend
    plt.subplot(2, 2, 2)
    plt.plot(metrics_df['date'], metrics_df['avg_response_time'])
    plt.title('Average Response Time Trend')
    plt.xlabel('Date')
    plt.ylabel('Response Time (s)')
    plt.xticks(rotation=45)
    
    # 3. User Ratings Distribution
    plt.subplot(2, 2, 3)
    sns.countplot(data=feedback_df, x='rating')
    plt.title('User Ratings Distribution')
    plt.xlabel('Rating')
    plt.ylabel('Count')
    
    # 4. Daily Prediction Volume
    plt.subplot(2, 2, 4)
    plt.plot(metrics_df['date'], metrics_df['total_predictions'])
    plt.title('Daily Prediction Volume')
    plt.xlabel('Date')
    plt.ylabel('Number of Predictions')
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()

# Create dashboard
create_monitoring_dashboard()

## 3. Retraining Pipeline

In [ ]:
class ModelRetrainer:
    def __init__(self, monitor):
        """Initialize retrainer with monitor."""
        self.monitor = monitor
        self.model_dir = Path('../models')
        
    def should_retrain(self, min_feedback=100, max_confidence_drop=0.1):
        """Check if model should be retrained."""
        # Load metrics
        with open(self.monitor.metrics_file, 'r') as f:
            metrics_data = json.load(f)
        
        metrics_df = pd.DataFrame(metrics_data['daily_metrics'])
        
        # Check feedback volume
        total_feedback = metrics_df['feedback_count'].sum()
        if total_feedback < min_feedback:
            return False
        
        # Check confidence trend
        recent_confidence = metrics_df['avg_confidence'].tail(7).mean()
        baseline_confidence = metrics_df['avg_confidence'].head(7).mean()
        
        if (baseline_confidence - recent_confidence) > max_confidence_drop:
            return True
        
        return False
    
    def prepare_retraining_data(self):
        """Prepare data for retraining."""
        # Load feedback data
        feedback_df = pd.read_csv(self.monitor.feedback_file)
        predictions_df = pd.read_csv(self.monitor.predictions_file)
        
        # Merge feedback with predictions
        retraining_data = predictions_df.merge(
            feedback_df,
            left_index=True,
            right_on='prediction_id',
            how='inner'
        )
        
        # Filter high-quality feedback
        retraining_data = retraining_data[retraining_data['rating'] >= 4]
        
        return retraining_data
    
    def retrain_model(self):
        """Retrain model with new data."""
        print("Checking retraining criteria...")
        
        if not self.should_retrain():
            print("Retraining criteria not met")
            return
        
        print("Preparing retraining data...")
        new_data = self.prepare_retraining_data()
        
        # Load current model
        current_model = joblib.load(self.model_dir / 'best_model.joblib')
        
        print(f"Starting retraining with {len(new_data)} new samples...")
        
        # TODO: Implement actual retraining
        # This would involve:
        # 1. Combining new data with original training data
        # 2. Retraining the model
        # 3. Evaluating performance
        # 4. Saving if better than current model
        
        print("Retraining complete")

# Initialize retrainer
retrainer = ModelRetrainer(monitor)

# Check retraining
retrainer.retrain_model()

## 4. Schedule Maintenance Tasks

In [ ]:
def schedule_maintenance():
    """Schedule regular maintenance tasks."""
    # This would typically be done using a task scheduler like cron
    # For demonstration, we'll just list the tasks
    
    maintenance_schedule = {
        'daily': [
            'Compute performance metrics',
            'Check for alerts',
            'Update dashboard'
        ],
        'weekly': [
            'Analyze prediction patterns',
            'Review user feedback',
            'Check retraining criteria'
        ],
        'monthly': [
            'Full model evaluation',
            'Data quality assessment',
            'System health check'
        ]
    }
    
    print("Maintenance Schedule:")
    for frequency, tasks in maintenance_schedule.items():
        print(f"\n{frequency.title()} Tasks:")
        for task in tasks:
            print(f"- {task}")

schedule_maintenance()